# Presentation of the code

Import package fipy: ('\*'means import all functions). 

Import random for the initial condition of the interface.

In [1]:
from fipy import *
import random

## Parameters

In [2]:
U = 0.8 #General speed
Mobility = 0.5 #ratio of the two viscosities; M_c in Hamouda's paper
epsilon =1. #Must be more than 2*dx
l = 1. #this is lambda from Hamouda's paper

## Geometry and mesh

We first install the geometry of the situation.
The simulation is 2D (W=1., L=1.).

### Space

In [3]:
L = 1. #length
W = 1. #width: characteristic length
b = 1. #gap

### Mesh

The mesh is decided with the length of one controle volume.

In [5]:
dx = 0.25 #width of controle volume
dy = 0.10
nx = 100
ny = 250 #number of controle volume
mesh = Grid2D(dx=dx, dy=dy, nx=nx, ny=ny)

## Description of the fluids

### Parameters

The cause of fingering is the ratio of the two viscosities. We can ste the second viscosity to one, to stay dimensionless.

Fluid 1 is the active fluid, less viscous.

Fluid 2 is the passive fluid, more viscous.  

The two permeabilities are set to 1.

We introduce the parameter beta, present in the model's formulas.

In [6]:
viscosity2 = 1.
M = Mobility * epsilon**2 #M in Hamouda's paper
viscosity1 = viscosity2 * Mobility
permeability1 = permeability2 = 1.
beta1 = viscosity1 / permeability1
beta2 = viscosity2 / permeability2

### Variable of the fluids

The grid is colocated in FiPy. We need to introduce a correction in the velocity to suppress oscillations. Therefore we need two variables of speed.

We have here two coupled equations: in order to the variables to respect both at each time step, we need an iterative scheme with a correction of the pressure at each iteration. For this correction, we use the SIMPLE algorithm.
We introduce a new pressure variable that corresponds to the pressure once it has been corrected by the second equation, that will be used to begin again the iteration.

In [7]:
pressure = CellVariable(mesh=mesh, name='pressure')
pressureCorrection = CellVariable(mesh=mesh)
xVelocity = CellVariable(mesh=mesh, name='X Velocity')
yVelocity = CellVariable(mesh=mesh, name='Y Velocity')
velocity = FaceVariable(mesh=mesh, rank=1)

## Phase-field model

### Order parameter

The order parameter is phi. The hasOld option permits to pply an iterative scheme on the reolution of the phi equation without advancing in time step. Because the phase-field equation is not linear we need to iterate the solving of the equation at each time step. The commad hasOld permits to update the value of phi to use at the end of the iterations only.

In [8]:
phi = CellVariable(name=r'$\phi$', mesh=mesh, hasOld=1)

### Phase-field equation

The first line is the creation of an interpolation of the value of phi on to the faces. FiPy would have it done automatically, but it is more accurate to do it by arithmeticFaceValue (We also get to choose what type of interpolation we use) (why arithmetic?? I think because more precise is not needed, but I need to review, not the priority though)

The equation chosen for the phase-field is base on Cahn-Hilliard model as described in Hamouda's paper.

In [11]:
PHI = phi.arithmeticFaceValue #result more accurate by non-linear interpolation
coeff1 = Mobility * l * (6.* PHI*(PHI-1.) + 1.)
eq = (TransientTerm() + ConvectionTerm(velocity) == DiffusionTerm(coeff=coeff1) - DiffusionTerm(coeff=(M, l)))

## Boundary conditions

### Phase

The first line is the outflow condition on the right.

'x,y = mesh.cellCenters' is to define x as the first axis so we can define the boundary conditions.

Phi is set up as a crenau function in the beginning. We set up a random initial condition in the position of the front phase so that we have a small random perturbation of the interface that permits the formation of fingers.
The first fluid is defined for phi=0. The second one is defined for phi=1.

In [15]:
phi.faceGrad.constrain([0.], mesh.facesRight)

x,y = mesh.cellCenters
def initialize(phi):
    phi.setValue(0.)
    for i in range(ny):
        a = random.gauss(0.1, 0.1*dx)
        phi.setValue(1., where=(x > nx*dx * a ) & (y<(i+1)*dy) & (y>(i*dy)))

initialize(phi)

### New values

The phase-field method means that the two fluids are not separated anymore: this means all the variable have to be defined on the whole domain. The particular variables are transformed by interpolation.

In [9]:
beta = CellVariable(mesh=mesh, name=r'$\beta$', value = beta2 * phi + beta1 * (1-phi), hasOld=1)

## Pressure and velocity

### Equations

We then introduce the equations: equation of motion, here Darcy's law and equation of continuity, permitting the correction of the pressure.

In [1]:
xVelocityEq = (ImplicitSourceTerm(coeff=beta) + pressure.grad.dot([1.,0.]))
yVelocityEq = (ImplicitSourceTerm(coeff=beta) + pressure.grad.dot([0.,1.]))

NameError: name 'ImplicitSourceTerm' is not defined

We then define the variables necessary for the correction for the double equations.

In [13]:
ap = CellVariable(mesh=mesh, value=1.)
coeff = 1./ ap.arithmeticFaceValue * mesh._faceAreas * mesh._cellDistances
pressureCorrectionEq = DiffusionTerm(coeff=coeff) - velocity.divergence

### Rhie-Chow correction

FiPy uses a colocated grid, causing oscillations in the velocity field. It is necessary to apply a correction called the Rhie-Chow correction.

In [14]:
from fipy.variables.faceGradVariable import _FaceGradVariable
volume = CellVariable(mesh=mesh, value=mesh.cellVolumes, name='Volume')
contrvolume=volume.arithmeticFaceValue

## Viewers

In [ ]:
viewer = Viewer(vars = (phi,), datamin=0., datamax=1.)
viewer2 = Viewer(vars = (xVelocity, yVelocity), datamin=0., datamax=1.)
viewer3 = Viewer(vars = (pressure), datamin=0, datamax=250.)

## Initialization

We need to initialize the phase-field to separate the effect of its formation to the dynamics of the problem. For this, we need to have the initial fields of the pressure and the velocity.

### Phase field

The time step here is only used to resolve the equation. There are not time steps, but iterations to get a more accurate solution: by using the new value obtained at the solving of the equation, we are closer to the solution, so by solving it again with this value, we get a smaller residual, a more accurate solution. (we have a discretized solution, so the closer the values are when solving, the solution gets more accurate : by convergence.)

In [17]:
dexp = 1.
elapsed = 0.
duration = 1500.
while elapsed < duration:
    phi.updateOld()
    dt = min(100, numerix.exp(dexp))
    elapsed += dt
    dexp += 0.01
    eq.solve(var=phi, dt = dt)
    if __name__ == '__main__':
        viewer.plot()

KeyboardInterrupt: 

### Pressure and velocity fields

We use the SIMPLE algorithm that we will use at each iteration.
The relaxation are kept the same as in the example.

In [ ]:
pressureRelaxation = 0.8
velocityRelaxation = 0.5
xVelocity.constrain(U, mesh.facesLeft)
yVelocity.constrain(0, mesh.facesTop | mesh.facesBottom)
pressureCorrection.constrain(0., mesh.facesRight)

sweeps = 41
for sweep in range(sweeps):
    ##Solve the Stokes equations to get starred value
    xVelocityEq.cacheMatrix()
    xres = xVelocityEq.sweep(var=xVelocity, underRelaxation=velocityRelaxation)
    xmat = xVelocityEq.matrix
    yres = yVelocityEq.sweep(var=yVelocity, underRelaxation=velocityRelaxation)
    ##update the ap coefficient from the matrix diagonal
    ap[:] = xmat.takeDiagonal()
    ##update the face velocities based on starred values with the Rhi-Chow correction
    #cell pressure gradient
    presgrad = pressure.grad
    #face pressure gradient
    facepresgrad = _FaceGradVariable(pressure)
    #
    velocity[0] = xVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[0].arithmeticFaceValue-facepresgrad[0])
    velocity[1] = yVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[1].arithmeticFaceValue-facepresgrad[1])
    velocity[0, mesh.facesLeft.value] = U
    velocity[0, mesh.facesRight.value] = U
    ##solve the pressure correction equation
    pressureCorrectionEq.cacheRHSvector()
    pres = pressureCorrectionEq.sweep(var=pressureCorrection)
    rhs = pressureCorrectionEq.RHSvector
    ## update the pressure using the corrected value
    pressure.setValue(pressure + pressureRelaxation * pressureCorrection)
    ## update the velocity using the corrected pressure
    xVelocity.setValue(xVelocity - pressureCorrection.grad[0] / ap * mesh.cellVolumes)
    yVelocity.setValue(yVelocity - pressureCorrection.grad[1] / ap * mesh.cellVolumes)
    xVelocity[0]=U
    if sweep%10 == 0:
        viewer2.plot()


viewer3.plot(filename="pressureini .png")
TSVViewer(vars=(pressure)).plot(filename="essaidonneini.tsv")

## Dynamics

In [ ]:
displacement = 50.
timeStep = 0.8 * dx / U #less than one space step per time step
elapsed = 0.

while elapsed < displacement/U:
    phi.updateOld()
    res = 1e+10
    while res > 1e-6:
        res = eq.sweep(var=phi, dt=timeStep)
    for sweep in range(sweeps):
        ##Solve the Stokes equations to get starred value
        xVelocityEq.cacheMatrix()
        xres = xVelocityEq.sweep(var=xVelocity, underRelaxation=velocityRelaxation)
        xmat = xVelocityEq.matrix
        yres = yVelocityEq.sweep(var=yVelocity, underRelaxation=velocityRelaxation)
        ##update the ap coefficient from the matrix diagonal
        ap[:] = xmat.takeDiagonal()
        ##update the face velocities based on starred values with the Rhi-Chow correction
        #cell pressure gradient
        presgrad = pressure.grad
        #face pressure gradient
        facepresgrad = _FaceGradVariable(pressure)
        #
        velocity[0] = xVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[0].arithmeticFaceValue-facepresgrad[0])
        velocity[1] = yVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[1].arithmeticFaceValue-facepresgrad[1])
#        velocity[0, mesh.facesLeft.value] = U
#        velocity[0, mesh.facesRight.value] = U
        ##solve the pressure correction equation
        pressureCorrectionEq.cacheRHSvector()
        pres = pressureCorrectionEq.sweep(var=pressureCorrection)
        rhs = pressureCorrectionEq.RHSvector
        ## update the pressure using the corrected value
        pressure.setValue(pressure + pressureRelaxation * pressureCorrection)
        ## update the velocity using the corrected pressure
        xVelocity.setValue(xVelocity - pressureCorrection.grad[0] / ap * mesh.cellVolumes)
        yVelocity.setValue(yVelocity - pressureCorrection.grad[1] / ap * mesh.cellVolumes)
#        xVelocity[0]=U
    elapsed +=timeStep
    viewer.plot()
    viewer2.plot()
    beta.setValue(beta2 * phi + beta1 * (1.-phi))
    if elapsed%5==0:
        viewer3.plot(filename="pressure%d .png" % elapsed)
        TSVViewer(vars=(pressure)).plot(filename="essaidonne%d .tsv" % elapsed)

This line is to save the data created.

In [ ]:
TSVViewer(vars=(phi, xVelocity, yVelocity, pressure)).plot(filename="essaidonne.tsv")

This last line stops the code at the end so we are able to see the results.

In [ ]:
raw_input("pause")